# Error Handling

The following "Order Fulfilment" process model orchestrates three microservices which are defined below. However, there is a 50/50 chance that the request for payment or fetching goods will fail.

![](/images/Error-Event-Sub.png)

### Tasks
Instruction to run/do the exercise:
1. Download the initial process model (Error-Event-Sub-Initial.bpmn in the modelling folder) and open it in the Camunda Modeler.
2. Extend the model as shown above in the final model using error events and compensations.
3. Deploy the process model with the Camunda Modeler to the classroom instance with your **tenant**.
4. Make sure that you have the correct Camunda engine URL:

In [1]:
camunda_eninge_rest = 'https://digibp.herokuapp.com/engine-rest'

5. Run the process model from the the Camunda Modeler.
6. Make sure that you use the right tenant:

In [2]:
tenant = 'showcase'

7. Run the notebook in Deepnote.
8. Investigate the process instantiated in the Camunda Cockpit and inspect the Deepnote output.

## Topic Subscription of (sometimes) failing Microservices

In the following, three microservices are imitated, each of which has a subscription of a topic command to be received. After a message is received, the task is completed. Here, a [custom-developed Camunda client worker implementation in Python](https://github.com/DigiBP/digibp-camunda-external-python-task/blob/master/cam.py) is used, which encapsulates the [external client REST API](https://docs.camunda.org/manual/latest/reference/rest/external-task/) of Camunda.

There is a random 50/50 chance that `retrievePayment` or `fetchGoods` are failing by sending a BPMN error.

In [3]:
import httpimport, random

with httpimport.github_repo("DigiBP", "digibp-camunda-external-python-task"):
    import cam

# Initialize worker client
worker = cam.Client(camunda_eninge_rest)

# Payment microservice callback function
def retrievePayment(taskid, response):
    if random.randint(0, 1):
        print("Payment Done")
        worker.complete(taskid)
    else:
        print("PaymentFailed")
        worker.error(taskid, "PaymentFailed")


# Refund payment microservice callback function
def refundPayment(taskid, response):
    print("Refund Payment Done")
    worker.complete(taskid)


# Inventory microservice callback function
def fetchGoods(taskid, response):
    if random.randint(0, 1):
        print("Goods Fetched")
        worker.complete(taskid)
    else:
        print("OutOfStock")
        worker.error(taskid, "OutOfStock")


# Shipment microservice callback function
def shipGoods(taskid, response):
    print("Goods Shipped")
    worker.complete(taskid)


# Subscriptions on topics
worker.subscribe("retrievePayment", retrievePayment, tenant)
worker.subscribe("refundPayment", refundPayment, tenant)
worker.subscribe("fetchGoods", fetchGoods, tenant)
worker.subscribe("shipGoods", shipGoods, tenant)

# Run polling server
worker.polling()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=002e5c7d-14b9-4bea-9ee8-dba256b85cbc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>